<!--
#  Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved.
#
#    Licensed under the Apache License, Version 2.0 (the "License").
#    You may not use this file except in compliance with the License.
#    You may obtain a copy of the License at
#
#        http://www.apache.org/licenses/LICENSE-2.0
#
#    Unless required by applicable law or agreed to in writing, software
#    distributed under the License is distributed on an "AS IS" BASIS,
#    WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#    See the License for the specific language governing permissions and
#    limitations under the License.
-->

# This notebook creates an image with additional Applications that can be used alongside Jupyter

## Content
1. [Configuration](#Configuration)
2. [Build Image](#Build-Image)


### Configuration

In [1]:
image_name = 'jupyter-with-apps'
folder_name = '/home/jovyan/shared/samples/notebooks/M-Admin/extra_user_apps/'

In [ ]:
%cd $folder_name

### Build Image

Lets see how  orbit build image works...

get our orbit env and team names

In [ ]:
env_name = %env AWS_ORBIT_ENV
team_name = %env AWS_ORBIT_TEAM_SPACE
user_name = %env USERNAME
namespace = %env AWS_ORBIT_USER_SPACE
(env_name,team_name, user_name, namespace)

Repository name will be created from the image name prefixed by the env context.  Users are only able to manipulate ECR repos that start with 'orbit-{env_name}/users/'

In [ ]:
repository_name = (f"orbit-{env_name}/users/{image_name}")
repository_name

In [ ]:
!aws ecr delete-repository --repository-name $repository_name --force

adding a file to our docker as an example

In [ ]:
pwd = %pwd
pwd

Now lets run the command

In [ ]:
%%time

output = !orbit build image -e $env_name -d $pwd -n $image_name 
output

Lets get the image address from the output of the previous command

In [ ]:
look_for = 'ECR Image Address='
image = None
for o in output:
    if look_for in o:
        image = o[o.index(look_for) + len(look_for):]
        print(image)

assert(image != None)       
    

In [ ]:
# check that the image was built
import json
print(repository_name)
images = !aws ecr list-images --repository-name $repository_name
images = "".join(images)
im = json.loads(images)
print(im['imageIds'])
assert(len(im['imageIds']) > 0)

### Building the configuration for the image

In [2]:
import json

customname = "orbit-custom-image-with-apps-"+team_name

podsetting = {
    "apiVersion": "orbit.aws/v1",
    "kind": "PodSetting",
    "metadata": {
        "labels": {
            "orbit/env": env_name,
            "orbit/space": "team",
            "orbit/team": team_name
        },
 
        "name": customname,
        "namespace": team_name,
    },
    "spec": {
        "containerSelector": {
            "jsonpath": "metadata.labels.app"
        },
        "desc": "Custom Image Example "+customname ,
        "env": [
            {
                "name": "custom_name",
                "value": "custom_value"
            }
        ],
        "podSelector": {
            "matchExpressions": [
                {
                    "key": "orbit/"+customname,
                    "operator": "Exists"
                },
                {
                    "key": "app",
                    "operator": "NotIn",
                    "values": [
                        "orbit-runner"
                    ]
                }
            ]
        },
        "resources": {
            "limits": {
                "cpu": "2.0",
                "memory": "2Gi"
            },
            "requests": {
                "cpu": "2.0",
                "memory": "2Gi"
            }
        },
        "securityContext": {
            "runAsUser": 1000
        },
        "image": image
    }
}


with open("podsetting.json", 'w') as f:
    json.dump(podsetting, f)

In [ ]:
!cat podsetting.json

In [ ]:
!kubectl apply -f podsetting.json -n {team_name}

## Users can now use the new profile to use new apps

***